<a href="https://colab.research.google.com/github/SnikhilK/ML_Learn/blob/main/Colorization_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import Activation, Dense, Dropout, Flatten, InputLayer
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf

config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
tf.keras.backend.compat.v1.set_session(tf.Session(config=config));

In [5]:
tf.config.list_physical_devices('GPU') 

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [12]:
os.listdir

<function posix.listdir>

In [15]:
os.getcwd()

'/content'

In [17]:
# Get images
X = []
for filename in os.listdir('/content/Train'):
    X.append(img_to_array(load_img('/content/Train/'+filename)))
X = np.array(X, dtype=float)

# Set up train and test data
split = int(0.95*len(X))
Xtrain = X[:split]
Xtrain = 1.0/255*Xtrain

In [18]:
model = Sequential()
model.add(InputLayer(input_shape=(256, 256, 1)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.compile(optimizer='rmsprop', loss='mse')

In [26]:
# Image transformer
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

# Generate training data
batch_size = 100000
def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)

In [27]:
# Train model      
tensorboard = TensorBoard(log_dir="output/first_run")

In [28]:
model.fit(image_a_b_gen(batch_size), epochs=10, steps_per_epoch=250)

Epoch 1/10
250/250 [==============================] - 139s 557ms/step - loss: 0.0082
Epoch 2/10
250/250 [==============================] - 139s 555ms/step - loss: 0.0056
Epoch 3/10
250/250 [==============================] - 138s 552ms/step - loss: 0.0038
Epoch 4/10
250/250 [==============================] - 137s 548ms/step - loss: 0.0028
Epoch 5/10
250/250 [==============================] - 137s 547ms/step - loss: 0.0024
Epoch 6/10
250/250 [==============================] - 137s 547ms/step - loss: 0.0020
Epoch 7/10
250/250 [==============================] - 137s 549ms/step - loss: 0.0015
Epoch 8/10
250/250 [==============================] - 137s 548ms/step - loss: 0.0014
Epoch 9/10
250/250 [==============================] - 137s 547ms/step - loss: 0.0011
Epoch 10/10
250/250 [==============================] - 137s 546ms/step - loss: 0.0012


In [29]:
# Save model
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")

In [36]:
# Test images
Xtest = rgb2lab(1.0/255*X[split:])[:,:,:,0]
Xtest = Xtest.reshape(Xtest.shape+(1,))
Ytest = rgb2lab(1.0/255*X[split:])[:,:,:,1:]
Ytest = Ytest / 128
print(model.evaluate(Xtest, Ytest, batch_size=batch_size))

1/1 [==============================] - 0s 21ms/step - loss: 0.0036
0.0035598445683717728


In [41]:
color_me = []
for filename in os.listdir('/content/Test/'):
    color_me.append(img_to_array(load_img('/content/Test/'+filename)))
color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))

# Test model
output = model.predict(color_me)
output = output * 128

In [42]:
# Output colorizations
for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = color_me[i][:,:,0]
    cur[:,:,1:] = output[i]
    imsave("Res/img_"+str(i)+".png", lab2rgb(cur))